In [2]:
%%time
import os
import json

import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.utils import unique_name_from_base

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = get_execution_role()
prefix = 'wenet'
output_path = f"s3://{bucket}/{prefix}"

print("boto3.__version__:{}".format(boto3.__version__))
print("sagemaker.__version__:{}".format(sagemaker.__version__))
print("bucket:{}".format(bucket))
print("role:{}".format(role))

boto3.__version__:1.26.8
sagemaker.__version__:2.116.0
bucket:sagemaker-us-east-1-348052051973
role:arn:aws:iam::348052051973:role/service-role/AmazonSageMakerServiceCatalogProductsExecutionRole
CPU times: user 1.23 s, sys: 925 ms, total: 2.16 s
Wall time: 1.03 s


In [3]:
%%markdown
Copy the wenet/examples/librispeech/s0/*.sh and wenet/examples/librispeech/s0/local to wenet/ as requested by Sagemaker
Overwrite the wenet/wenet/bin/train.py with the given one
Change the /root/wenet to /opt/ml/input in all data.list files (especially for train_960 and dev)
The "Librispeech" in data.list file in Github has the wrong captalization because it's wrong when I upload it. Please change it yourself!

Copy the wenet/examples/librispeech/s0/*.sh and wenet/examples/librispeech/s0/local to wenet/ as requested by Sagemaker
Overwrite the wenet/wenet/bin/train.py with the given one


In [15]:
%%time
instance_type = "ml.p3.16xlarge"

hyperparameters = {
    'datadir':'/opt/ml/input/data/training',
    'stage': '4',
    'stop_stage': '5',
    'train_config': 'examples/librispeech/s0/conf/train_conformer.yaml',
    'model_dir': '/opt/ml/model',
}

est = PyTorch(
    entry_point="run-8gpu.sh",
    source_dir="./wenet",
    framework_version="1.11.0",
    py_version="py38",
    role=role,
    instance_count=1,
    instance_type=instance_type,
    volume_size=200,
    disable_profiler=True,
    debugger_hook_config=False,
    base_job_name=prefix,
    hyperparameters = hyperparameters,
    keep_alive_period_in_seconds=1800,
)


CPU times: user 48.5 ms, sys: 16 ms, total: 64.6 ms
Wall time: 63.8 ms


In [16]:
from sagemaker.inputs import TrainingInput
prefix_dataset = "wenet/export"
loc =f"s3://{bucket}/{prefix_dataset}"

training = TrainingInput(
    s3_data_type='S3Prefix', # Available Options: S3Prefix | ManifestFile | AugmentedManifestFile
    s3_data=loc,
    distribution='FullyReplicated', # Available Options: FullyReplicated | ShardedByS3Key 
    input_mode='FastFile'
)

In [ ]:
%%time
job_name = est.fit({"training":training})
#job_name = est.fit()

2022-11-28 03:24:31 Starting - Starting the training job......
2022-11-28 03:25:29 Starting - Preparing the instances for training.........
2022-11-28 03:26:53 Downloading - Downloading input data......
2022-11-28 03:27:44 Training - Downloading the training image.................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-11-28 03:30:39,044 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-11-28 03:30:39,120 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-11-28 03:30:39,135 sagemaker_pytorch_container.training INFO     Invoking user training script.

2022-11-28 03:30:35 Training - Training image download completed. Training in progress.2022-11-28 03:30:50,654 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/

In [ ]:
model_data = est.model_data
print("Model artifact saved at:\n", model_data)

In [9]:
!cat wenet/requirements.txt

torchaudio==0.10.0
Pillow
pyyaml>=5.1
sentencepiece
tensorboard
tensorboardX
typeguard
textgrid
pytest
flake8==3.8.2
flake8-bugbear
flake8-comprehensions
flake8-executable
flake8-pyi==20.5.0
mccabe
pycodestyle==2.6.0
pyflakes==2.2.0